## Загрузка данных

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv('insurance.csv')

display(df.head())
display(df.info())
display(df.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [18]:
x = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

* **Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.
* **Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

Посмотрим, как признаки взаимосвязаны между собой. 

In [19]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


В целом, признаки мало связаны между собой, коэффицент корреляции у всех находится около нуля. Самую большую взаимосвязь имеет возраст со страховыми выплатами. Видимо, чем выше возраст клиента, тем вероятнее у него будет запись о страховой выплате.  

##### Вывод

Данные загружены и просмотрены, предобработку проводить не требуется, так как ошибок в данных не обнаружено. Определены признаки и целевой признак. 

## Умножение матриц

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

**Ответ:** Качество предсказаний модели не изменится.  

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y 
$$

**Обоснование:** 

$$
w' = ((XP^T) (XP))^{-1} (XP^T) y  = ((XP)^T(XP))^{-1} (XP)^Ty = P^{-1}((XP)^{T}X)^{-1} (XP)^Ty  = P^{-1}((XP)^{T}X)^{-1} P^TX^Ty  = P^{-1}(P^TX^TX)^{-1} P^TX^Ty = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty
$$


Произведем замену:
$$
P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w
$$

$$
w' = P^{-1}w
$$

Значит:

$$
a' = X'w'
$$

$$
a' = XPP^{-1}w
$$

$$
a' = Xw
$$

##### Вывод

$$
a = a'
$$ Качество линейной регрессии не изменится. 

## Алгоритм преобразования

**Алгоритм**

Умножим матрицу признаков на сгенерированную случайную обратимую матрицу.

**Обоснование**

Данные будут защищены, а качество линейной регрессии не изменится, так как Xw = X'w' - это мы выяснили в прошлом пункте. 

## Проверка алгоритма

In [20]:
def r2_score(x, y):
    lr = LinearRegression()
    lr.fit(x, y)
    r2 = lr.score(x, y)
    return r2

In [21]:
r2_score(x, y)

0.42494550286668

In [22]:
# Создание обратимой матрицы
r = np.random.RandomState(12345)

def inverse_matrix(r):
    inv_matrix = r.rand(4, 4)
    try:
        np.linalg.inv(inv_matrix)
        return inv_matrix
    except:
        return inverse_matrix(r)
        
        
inv_matrix = inverse_matrix(r)

Зашифруем данные. 

In [23]:
x_inv = x @ inv_matrix
r2_score(x_inv, y)

0.42494550286667643

Попробуем расшифровать данные. 

In [24]:
inv_x = x_inv @ np.linalg.inv(inv_matrix)
r2_score(inv_x, y)

0.424945502866673

## Общий вывод

В ходе работы мы ответили на вопрос об изменении качества модели машинного обучения и пришли к выводу, что от умножения признаков на обратную матрицу оно не изменится. На этом основании мы разработали алгоритм шифрования данных, который защитит информацию и не повлияет на качество модели. 